In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np 
from keras import backend as K
from transformers import AutoTokenizer

df = pd.read_csv("FinalBalancedDataset.csv") # 1 = toxic, 0 = not toxic
df.groupby('Toxicity').describe() #32592 not toxic, 24153 toxic, imbalanced dataset so I need to downsample, only take 24153 not toxic samples so we have balanced data inputs

dfToxic = df[df['Toxicity'] == 1] #originally had as '1', but it's an int not a character so you can just set to 1 
dfNotToxic = df[df['Toxicity'] == 0]

dfNotToxicDownsampled = dfNotToxic.sample(dfToxic.shape[0]) #assume that the data is shuffled (it shouldn't be in order and it wouldn't matter anyway), now has 24153 values

xTrain, xTest, yTrain, yTest = train_test_split(dfToxic, dfNotToxicDownsampled, test_size = 0.33, random_state = 42) #split data, stratify ensures distribution of categories is equal

# xTrain = tf.convert_to_tensor(xTrain, dtype = tf.string)
# yTrain = tf.convert_to_tensor(xTrain, dtype = tf.string)

bertPreprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bertEncoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

def getSentenceEmbedding(sentences): #basically turns sentences into tensors
    
    preprocessedText = bertPreprocess(sentences) #first you preprocess, then you encode
    return bertEncoder(preprocessedText)['pooled_output'] #encoding basically turns data into numbers

#defining layers
textInput = tf.keras.layers.Input(shape = (), dtype = tf.string, name = "text")

preprocessedText = bertPreprocess(textInput)
outputs = bertEncoder(preprocessedText)

layerOne = tf.keras.layers.Dropout(0.1, name = 'dropout')(outputs['pooled_output'])
layerTwo = tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'ouptut')(layerOne)

#define model
model = tf.keras.Model(inputs = [textInput], outputs = [layerTwo])

metrics = [
    tf.keras.metrics.BinaryAccuracy(name = 'accuracy'),
    tf.keras.metrics.Precision(name = 'precision'),
    tf.keras.metrics.Recall(name = 'recall')
]

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = metrics)

model.fit(xTrain, yTrain, epochs = 10)

#didn't work, need to pass in tokenized data, not string and then tokenize :(
    

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-06-04 09:43:35.293077: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-06-04 09:43:35.293320: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type int).